# Families to Ka/Ks values

This notebook needs as input a file containing sequences ID and their respective family ID
It will perform the Ka/Ks computation through the Paml programm

In [38]:
from Bio import Seq
from Bio import SeqIO
import subprocess
import re
import pandas as pd
import matplotlib.pyplot as plot

In [66]:
raw_seq_prot = list(SeqIO.parse("source_data/Theobroma_cacao.Theobroma_cacao_20110822.pep.all.fa", "fasta"))
raw_seq_cds = list(SeqIO.parse("source_data/Theobroma_cacao.Theobroma_cacao_20110822.cds.all.fa", "fasta"))

In [40]:
ceb = pd.read_table("results_from_galaxy_FTAG_finder/families_filter30-70.tabular", sep="\t")

In [13]:
# get unique family
families_set = set(ceb.family)
gene_id_by_family = list()

for family in families_set:
    a = list(ceb[ceb.family == family]["geneName"])
    gene_id_by_family.append(a)


In [18]:
# In order to reduce the amount of computation only one couple of sequence per family are considered. 
gene_id_by_family = [ x[0:2] for x in gene_id_by_family]

In [37]:
# check family length
# [len(x) for x in gene_id_by_family]
# seq_ID_to_write = list(ceb[ceb["family"] == target_family]["geneName"])
# seq_ID_to_write = list(ceb[0:500]["geneName"])

In [19]:
all_ka = list()
all_ks = list()

### Main loop

**First level**

For each family we get the IDs element which is a list of sequence ID.
Raw pep and cds files are read and sequences of targeted families are written in target_pep/cds.fa files

**Second level**

fasta files created in the first loop are read and each sequence inside is written one at a time in a tmp fasta file.
Of course this step is useless in the case where only one couple of sequence per family is used. However, it will be needed if the full Ka/Ks analysis need to be done.


The Paml script is then launched through a bash script. Than tmp files are erased in order not to crowd current folder

In [73]:

for IDs in gene_id_by_family:
    with open("target_pep.fa", "w") as OUT_pep:
        for seq in raw_seq_prot:
            if seq.id in IDs:
                SeqIO.write(seq, OUT_pep, "fasta")

    with open("target_cds.fa", "w") as OUT_cds:
        for seq in raw_seq_cds:
            if seq.id in IDs:
                SeqIO.write(seq, OUT_cds, "fasta")
        
    pep_list = list(SeqIO.parse("target_pep.fa", "fasta"))
    cds_list = list(SeqIO.parse("target_cds.fa", "fasta"))
    nb_seq = len(pep_list)
    
    for i in range(0, nb_seq-1):
        for j in range(i+1, nb_seq):
            with open("tmp_pep.fa", "w") as OUT_pep:
                SeqIO.write(pep_list[i], OUT_pep, "fasta")
                SeqIO.write(pep_list[j], OUT_pep, "fasta")
            with open("tmp_cds.fa", "w") as OUT_cds:
                SeqIO.write(cds_list[i], OUT_cds, "fasta")
                SeqIO.write(cds_list[j], OUT_cds, "fasta")

            cmd = "./paml_run.sh tmp_pep.fa tmp_cds.fa"
            subprocess.run(cmd, stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)

            # Line with Ks and Ka values in yn result file is the only one starting with 
            res_line = "".join([line for line in open('yn') if re.match('^\s+[0-9]',line)])
            res_line = re.split("\s+", res_line)
            all_ka.append(res_line[8])
            all_ks.append(res_line[11])
            subprocess.run("rm 2YN.* cds.ali.phy prot.ali.aln res rst rst1 rub tmp_pep.dnd yn tmp_cds.fa tmp_pep.fa ", 
                           stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)
    subprocess.run("rm target_pep.fa target_cds.fa ", stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)
    

In [42]:
# save all output (Ka, Ks and ratio of both) in a csv file
OUT_ka_ks = open("ka_ks_values", "w")
OUT_ka_ks.write("ka;ks;ratio\n")
for i in range(len(all_ka)):
        if float(all_ks[i]) == 0:
            ratio = 0
        else :
            ratio = float(all_ka[i]) / float(all_ks[i])
        OUT_ka_ks.write(str(all_ka[i]) + ";" + str(all_ks[i]) + ";" + str(ratio) + "\n" )
OUT_ka_ks.close()